处理用于 aspect level sentiment classification 的 laptop 和 restaurant 数据

In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [2]:
def clean(text):
    return text_to_word_sequence(text,
                                 filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                 lower=True, split=" ")

In [3]:
def do_process(input_frame, output_frame):
    with open(output_frame, 'w') as f:
        tree = ET.parse(input_frame)
        root = tree.getroot()
        sentence_num = 0
        aspect_num = 0
        data_dict = {
            'polarity': [],
            'aspect_term': [],
            'review_text': []
        }
        polarity_dict = {
            'positive': 0,
            'neutral': 1,
            'negative': 2
        }
        for sentence in root.iter('sentence'):
            sentence_num = sentence_num + 1
            text = sentence.find('text').text
            for asp_terms in sentence.iter('aspectTerms'):
                for asp_term in asp_terms.findall('aspectTerm'):
                    if asp_term.get('polarity') != 'conflict' and asp_term.get('term') != None:
                        aspect_num = aspect_num + 1
                        new_text = ''.join((text[:int(asp_term.get('from'))], 'aspect_term', text[int(asp_term.get('to')):]))
                        data_dict['review_text'].append(clean(new_text.strip()))
                        data_dict['aspect_term'].append(clean(asp_term.get('term')))
                        data_dict['polarity'].append(polarity_dict[asp_term.get('polarity')])
        print("Read %s sentences %s aspects" % (sentence_num, aspect_num))
        data = pd.DataFrame(data_dict)
        data.to_csv(output_frame)

In [4]:
input_frame1 = '../new_data/restaurant/train.xml'
output_frame1 = '../new_data/restaurant/train.csv'

input_frame2 = '../new_data/restaurant/test.xml'
output_frame2 = '../new_data/restaurant/test.csv'

input_frame3 = '../new_data/laptop/train.xml'
output_frame3 = '../new_data/laptop/train.csv'

input_frame4 = '../new_data/laptop/test.xml'
output_frame4 = '../new_data/laptop/test.csv'

In [6]:
do_process(input_frame1, output_frame1)

Read 3044 sentences 3608 aspects
